In [1]:
# Clean up
!hdfs dfs -rm -r -f /datasets /tmp
!rm -rf /tmp/hadoop_git_readme*
!hdfs dfs -expunge

16/05/10 19:34:19 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /tmp
16/05/10 19:34:22 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.


## Command line

In [2]:
!hdfs dfsadmin -report

Configured Capacity: 42241163264 (39.34 GB)
Present Capacity: 37536710656 (34.96 GB)
DFS Remaining: 37346992128 (34.78 GB)
DFS Used: 189718528 (180.93 MB)
DFS Used%: 0.51%
Under replicated blocks: 0
Blocks with corrupt replicas: 0
Missing blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 127.0.0.1:50010 (localhost)
Hostname: sparkbox
Decommission Status : Normal
Configured Capacity: 42241163264 (39.34 GB)
DFS Used: 189718528 (180.93 MB)
Non DFS Used: 4704452608 (4.38 GB)
DFS Remaining: 37346992128 (34.78 GB)
DFS Used%: 0.45%
DFS Remaining%: 88.41%
Configured Cache Capacity: 0 (0 B)
Cache Used: 0 (0 B)
Cache Remaining: 0 (0 B)
Cache Used%: 100.00%
Cache Remaining%: 0.00%
Xceivers: 1
Last contact: Tue May 10 19:34:23 UTC 2016




In [3]:
!hdfs dfs -ls /

Found 2 items
drwxr-xr-x   - vagrant supergroup          0 2016-05-10 19:05 /spark
drwxr-xr-x   - vagrant supergroup          0 2016-05-10 18:48 /user


In [4]:
!hdfs dfs -df -h /

Filesystem               Size     Used  Available  Use%
hdfs://localhost:9000  39.3 G  180.9 M     34.8 G    0%


In [5]:
!hdfs dfs -du -h /

179.0 M  /spark
473.4 K  /user


In [6]:
!hdfs dfs -mkdir /datasets

In [7]:
!wget -q http://www.gutenberg.org/cache/epub/100/pg100.txt \
    -O ../datasets/shakespeare_all.txt

In [8]:
!hdfs dfs -put ../datasets/shakespeare_all.txt \
    /datasets/shakespeare_all.txt

!hdfs dfs -put ../datasets/hadoop_git_readme.txt \
    /datasets/hadoop_git_readme.txt

In [9]:
!hdfs dfs -ls /datasets

Found 2 items
-rw-r--r--   1 vagrant supergroup       1365 2016-05-10 19:34 /datasets/hadoop_git_readme.txt
-rw-r--r--   1 vagrant supergroup    5589889 2016-05-10 19:34 /datasets/shakespeare_all.txt


In [10]:
!hdfs dfs -cat /datasets/hadoop_git_readme.txt | wc -l

30


In [11]:
!hdfs dfs -cat \
    hdfs:///datasets/hadoop_git_readme.txt \
    file:///home/vagrant/datasets/hadoop_git_readme.txt | wc -l

60


In [12]:
!hdfs dfs -cp /datasets/hadoop_git_readme.txt \
    /datasets/copy_hadoop_git_readme.txt

In [13]:
!hdfs dfs -rm /datasets/copy_hadoop_git_readme.txt

16/05/10 19:35:07 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /datasets/copy_hadoop_git_readme.txt


In [14]:
!hdfs dfs -expunge

16/05/10 19:35:09 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.


In [15]:
!hdfs dfs -get /datasets/hadoop_git_readme.txt \
    /tmp/hadoop_git_readme.txt

In [16]:
!hdfs dfs -tail /datasets/hadoop_git_readme.txt

ntry, of 
encryption software.  BEFORE using any encryption software, please 
check your country's laws, regulations and policies concerning the
import, possession, or use, and re-export of encryption software, to 
see if this is permitted.  See <http://www.wassenaar.org/> for more
information.

The U.S. Government Department of Commerce, Bureau of Industry and
Security (BIS), has classified this software as Export Commodity 
Control Number (ECCN) 5D002.C.1, which includes information security
software using or performing cryptographic functions with asymmetric
algorithms.  The form and manner of this Apache Software Foundation
distribution makes it eligible for export under the License Exception
ENC Technology Software Unrestricted (TSU) exception (see the BIS 
Export Administration Regulations, Section 740.13) for both object 
code and source code.

The following provides more details on the included cryptographic
software:
  Hadoop Core uses the SSL libraries from the Jetty project 

## Snakebite

In [17]:
from snakebite.client import Client
client = Client("localhost", 9000)

In [18]:
client.serverdefaults()

{'blockSize': 134217728L,
 'bytesPerChecksum': 512,
 'checksumType': 2,
 'encryptDataTransfer': False,
 'fileBufferSize': 4096,
 'replication': 1,
 'trashInterval': 0L,
 'writePacketSize': 65536}

In [19]:
for x in client.ls(['/']):
    print x['path']

/datasets
/spark
/user


In [20]:
client.df()

{'capacity': 42241163264L,
 'corrupt_blocks': 0L,
 'filesystem': 'hdfs://localhost:9000',
 'missing_blocks': 0L,
 'remaining': 37341663232L,
 'under_replicated': 0L,
 'used': 195353480L}

In [21]:
list(client.du(["/"]))

[{'length': 5591254L, 'path': '/datasets'},
 {'length': 187698038L, 'path': '/spark'},
 {'length': 484810L, 'path': '/user'}]

In [22]:
# Note:
# put command is not yet available

In [23]:
for el in client.cat(['/datasets/hadoop_git_readme.txt']):
    print el.next().count("\n")

30


In [24]:
# Note:
# copy command is not yet available

In [25]:
client.delete(['/datasets/shakespeare_all.txt']).next()

{'path': '/datasets/shakespeare_all.txt', 'result': True}

In [26]:
(client
.copyToLocal(['/datasets/hadoop_git_readme.txt'], 
             '/tmp/hadoop_git_readme_2.txt')
.next())

{'error': '',
 'path': '/tmp/hadoop_git_readme_2.txt',
 'result': True,
 'source_path': '/datasets/hadoop_git_readme.txt'}

In [27]:
list(client.mkdir(['/datasets_2']))

[{'path': '/datasets_2', 'result': True}]

In [28]:
list(client.delete(['/datasets*'], recurse=True))

[{'path': '/datasets', 'result': True},
 {'path': '/datasets_2', 'result': True}]